# Exercise - learning rate and batch size

1. Consider last exercise (i.e. the MNIST data). Suppose you are restricted to **training for only 2 epochs** but still want a good model. You recognize that finding the right learning rate is going to be very important. For this reason, you split your training data into a train and a validation set and use the validation set to find the optimal learning rate. Train a model with you optimized learning rate and evaluate it on your test data.
1. Recognizing that the batch size is also important for training speed, you decide to extend your above analysis to also find the optimal batch size. Once again, train a model with you optimized learning rate *and* batch size and evaluate it on your test data.
1. You have heard that momentum is important. You know that many optimizers already incorporate momentum by default, but you are now forced by your evil teacher to use SGD and otherwise repeat (1) and (2). You decide to extend your above analysis to also find the optimal momentum for SGD (see https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD for how to set it). Once again, train a model with you optimized learning rate, batch size, *and* momentum and evaluate it on your test data.

**See slides for more details!**

# Exercise 1

Consider last exercise (i.e. the MNIST data). Suppose you are restricted to **training for only 2 epochs** but still want a good model. You recognize that finding the right learning rate is going to be very important. For this reason, you split your training data into a train and a validation set and use the validation set to find the optimal learning rate. Train a model with you optimized learning rate and evaluate it on your test data.

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# A simply way to scale data. Remember pixels are in [0, 255] so this ensures [0, 1]
x_train = x_train / 255
x_test = x_test / 255

# Split train data to also get val data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape)

(48000, 28, 28) (48000,) (12000, 28, 28) (12000,) (10000, 28, 28) (10000,)


Here is (parts of) a model to get you started.

It is very helpful to wrap it inside a function since you want to call it multiple times in a loop.

Take note of the "Flatten" layer. This is important to reshape your data from (28, 28) to (784,).

Alternatively, you could reshape your data (the x's). This can be done using:

$\texttt{x = x.reshape(n, 784)}$ 

where $n$ is the number of samples (60k for training, 10k for test).

Then you don't need the Flatten layer, but remember to still specify an input shape of your first layer (i.e. 784 if you have done this reshaping).

**Note**: Do feel free to experiment with the number of layers, nodes per layer, and optimizer.

In [6]:
def build_model(learning_rate):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(16, activation='softmax'),
    ])
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        loss='SparseCategoricalCrossentropy',
        optimizer=optimizer,
        metrics=['accuracy'],
    )
    
    return model

Let us look at single run.

In [7]:
model = build_model(0.01) # insert desired learning rate
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2)
model.evaluate(x_test, y_test)

c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2
1500/1500 [==============================] - 5s 2ms/step - loss: 0.2908 - accuracy: 0.9143 - val_loss: 0.2092 - val_accuracy: 0.9433
Epoch 2/2
313/313 [==============================] - 1s 2ms/step - loss: 0.1842 - accuracy: 0.9491


[0.18419645726680756, 0.9491000175476074]

Now for the optimization.

In [9]:
learning_rates = [0.001,0.01,1,2] # must be positive floats. Default depends on optimizer

results = []

for learning_rate in learning_rates:
    model = build_model(learning_rate)
    model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, verbose=0) # you may want to use verbose=0 here to not get spammed. Also remember to use epochs=2
    loss, acc = model.evaluate(x_test,y_test)
    results.append((acc, learning_rate))
    
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate'])
results

c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


313/313 [==============================] - 1s 1ms/step - loss: 2.5157 - accuracy: 0.0958


,Accuracy,Learning rate
0,0.9591,0.001
1,0.9506,0.010
2,0.1135,1.000
3,0.0958,2.000


In [10]:
results[results['Accuracy'] == results['Accuracy'].max()]

,Accuracy,Learning rate
0,0.9591,0.001


In [17]:
# Train and evaluate final model.
# Remember to use both train and val data for training for best performance! 
# Similar to what we have done in all the other exercises/assignments
learning_rate = 0.001
results = []
model = build_model(learning_rate)
model.fit(x_train, y_train, epochs=2, verbose=0) # you may want to use verbose=0 here to not get spammed. Also remember to use epochs=2
loss, acc = model.evaluate(x_test,y_test)
results.append((acc,learning_rate))



c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


313/313 [==============================] - 1s 2ms/step - loss: 0.1482 - accuracy: 0.9548


In [19]:
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate'])
print(results)

   Accuracy  Learning rate
0    0.9548          0.001


# Exercise 2

Recognizing that the batch size is also important for training speed, you decide to extend your above analysis to also find the optimal batch size. Once again, train a model with you optimized learning rate *and* batch size and evaluate it on your test data.

In [22]:
learning_rates = [0.001,0.01,0.1] # must be positive floats. Default depends on optimizer
batch_sizes = [16, 32,64 ,128] # # must be positive ints. Default is 32

results = []

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        model = build_model(learning_rate)
        model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, verbose=0, batch_size=batch_size) # remember to pass in batch_size here! Also remember to use epochs=2
        loss, acc = model.evaluate(x_test,y_test)
        results.append((acc, learning_rate, batch_size))
    
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate', 'Batch size'])
results

c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


313/313 [==============================] - 1s 2ms/step - loss: 0.6130 - accuracy: 0.8229


,Accuracy,Learning rate,Batch size
0,0.9593,0.001,16
1,0.9598,0.001,32
2,0.9483,0.001,64
3,0.9503,0.001,128
4,0.9377,0.010,16
5,0.9435,0.010,32
6,0.9513,0.010,64
7,0.9502,0.010,128
8,0.1552,0.100,16
9,0.2976,0.100,32


In [23]:
results[results['Accuracy'] == results['Accuracy'].max()]

,Accuracy,Learning rate,Batch size
1,0.9598,0.001,32


In [ ]:
# Train and evaluate final model.
# Remember to use both train and val data for training for best performance! 
# Similar to what we have done in all the other exercises/assignments

# Exericse 3

You have heard that momentum is important. You know that many optimizers already incorporate momentum by default, but you are now forced by your evil teacher to use SGD and otherwise repeat (1) and (2). You decide to extend your above analysis to also find the optimal momentum for SGD (see https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD for how to set it). Once again, train a model with you optimized learning rate, batch size, *and* momentum and evaluate it on your test data.

In [ ]:
def build_model_with_momentum(learning_rate, momentum):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(??, activation=??),
        tf.keras.layers.Dense(??, activation=??),
        tf.keras.layers.Dense(??, activation=??),
    ])
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum)
    model.compile(
        loss=??,
        optimizer=optimizer,
        metrics=['accuracy'],
    )
    
    return model

In [ ]:
learning_rates = [] # must be positive floats. Default (for SGD) is 0.01
batch_sizes = [] # # must be positive ints. Default is 32
momentums = [] # must be in [0, 1). Default (for SGD) is 0.0

results = []

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for momentum in momentums:
            model = build_model_with_momentum(learning_rate, momentum=momentum)
            model.fit(??) # Remember to use epochs=2
            loss, acc = model.evaluate(??)
            results.append((acc, learning_rate, batch_size, momentum))
    
results = pd.DataFrame(results, columns=['Accuracy', 'Learning rate', 'Batch size', 'Momentum'])
results

In [ ]:
results[results['Accuracy'] == results['Accuracy'].max()]

In [ ]:
# Train and evaluate final model.
# Remember to use both train and val data for training for best performance! 
# Similar to what we have done in all the other exercises/assignments